In [1]:
from scripts.utils.file_utils import load_json, dump_json
from scripts.utils.helpers import get_points_in_poly
from scripts.utils.helpers import haversine, is_within_cirle
from scripts.traceroute_measurements.traceroutes import get_rtt_diff_id, get_probes_to_use_for_circles
from default import ANALYZABLE_FILE, REMOVED_PROBES_FILE, ANCHORS_FILE

## Remove bad results

In [13]:
removed_probes = load_json(REMOVED_PROBES_FILE)
all_probes = load_json(ANCHORS_FILE)

bad_targeted_ip = []
bad_source = {}

for probe in all_probes:
    if probe["address_v4"] in removed_probes:
        bad_targeted_ip.append(all_probes[probe]["address_v4"])
        lat = all_probes[probe]["geometry"]["coordinates"][1]
        lon = all_probes[probe]["geometry"]["coordinates"][0]
        bad_source[(lat, lon)] = all_probes[probe]["address_v4"]

In [ ]:
all_results = load_json(ANALYZABLE_FILE)
print(f"{len(all_results)} done results")

to_redo = []
removed_without_redo = 0
good_results = {}
bad_vp_count = 0
tier1_failed_count = 0
cbg_failed_count = 0
without_points_info = 0
no_negative_rtt = 0

for k, dst in all_results.items():
    if k in bad_targeted_ip:
        removed_without_redo += 1
        continue
    bvp = False
    for vp in dst['vps']:
        if (vp[0], vp[1]) in bad_source:
            bvp = True
    if bvp:
        to_redo.append(k)
        bad_vp_count += 1
        continue
    if not dst['tier1:done']:
        to_redo.append(k)
        tier1_failed_count += 1
        continue
    all_in = True
    candidate_geo = (dst['lat_c'], dst['lon_c'])
    for vp in dst['vps']:
        if not is_within_cirle((vp[0], vp[1]), vp[2], candidate_geo, speed_threshold=4/9):
            all_in = False
    if not all_in:
        to_redo.append(k)
        cbg_failed_count += 1
        continue
    if 'tier2:inspected_points_count' not in dst and 'tier3:inspected_points_count' not in dst:
        to_redo.append(k)
        without_points_info += 1
        continue
    if 'negative_rtt_included' not in dst:
        to_redo.append(k)
        no_negative_rtt += 1
        continue

    good_results[k] = dst

print(f"{removed_without_redo} result we can remove without redo")
print(f"{len(to_redo)} targets to redo")
print(f"{bad_vp_count} to redo because of miss placed vp")
print(f"{tier1_failed_count} to redo because tier1 failed")
print(f"{cbg_failed_count} to redo because cbg got the wrong area")
print(f"{without_points_info} to redo because old without points info")
print(f"{no_negative_rtt} to redo because old without negative rtt")
print(f"{len(good_results)} result to keep")

dump_json(good_results, ANALYZABLE_FILE)

## Remove bad CBG

In [2]:
all_results = load_json(ANALYZABLE_FILE)
print(f"{len(all_results)} done results")

to_redo = []
good_results = {}
tier1_failed_count = 0
bad_cbg = 0

for k, dst in all_results.items():
    if not dst['tier1:done']:
        to_redo.append(k)
        tier1_failed_count += 1
        continue
    else:
        tmp_cercles = []
        for vp in dst['vps']:
            tmp_cercles.append((vp[0], vp[1], vp[2], None, None))

        points = get_points_in_poly(tmp_cercles, 36, 5, 4/9)
        if len(points) > 0:
            p1 = points[0]
            p2 = (dst['tier1:lat'], dst['tier1:lon'])
            if haversine(p1, p2) > 0.01:
                bad_cbg += 1
                continue
        else:
            print("BAAADD")
    good_results[k] = dst

print(f"{bad_cbg} to redo because cbg failed")
print(f"{tier1_failed_count} to redo because tier1 failed")
print(f"{len(good_results)} result to keep")

dump_json(good_results, ANALYZABLE_FILE)


350 done results
3 to redo because cbg failed
0 to redo because tier1 failed
347 result to keep


## Fill negative rtt

In [4]:
res = load_json(ANALYZABLE_FILE)
print(len(res))
 
all_res = {}
for k, v in res.items():
    if 'negative_rtt_included' not in v:
        continue

    all_res[k] = v
print(len(all_res))

dump_json(all_res, ANALYZABLE_FILE)

347
347


In [5]:
all_res = load_json(ANALYZABLE_FILE)

i = 0
for ip, d in all_res.items():
    if 'negative_rtt_included' in d and d['negative_rtt_included']:
        continue
    i += 1
    probes = get_probes_to_use_for_circles(d['vps'])
    probe_ips = []
    for p in probes:
        probe_ips.append(p['address_v4'])
    if 'tier2:traceroutes' in d and 'tier2:landmarks' in d:
        landmark_ips = []
        landmarks = d['tier2:landmarks']
        for l in landmarks:
            landmark_ips.append(l[0])
        target_ip = d['target_ip']
        traceroutes = []
        for p in probe_ips:
            for l in landmark_ips:
                rtt, r1ip, traceroute_id = get_rtt_diff_id(p, target_ip, l)
                for landmark in landmarks:
                    if landmark[0] == l:
                        traceroutes.append([p, target_ip, l, r1ip, rtt, landmark[2], landmark[3], traceroute_id])
                        break
        all_res[ip]['tier2:traceroutes'] = traceroutes

    if 'tier3:traceroutes' in d and 'tier3:landmarks' in d:
        landmark_ips = []
        landmarks = d['tier3:landmarks']
        for l in landmarks:
            landmark_ips.append(l[0])
        target_ip = d['target_ip']
        traceroutes = []
        for p in probe_ips:
            for l in landmark_ips:
                rtt, r1ip, traceroute_id = get_rtt_diff_id(p, target_ip, l)
                for landmark in landmarks:
                    if landmark[0] == l:
                        traceroutes.append([p, target_ip, l, r1ip, rtt, landmark[2], landmark[3], traceroute_id])
                        break
        all_res[ip]['tier3:traceroutes'] = traceroutes
    all_res[ip]['negative_rtt_included'] = True
print(f"{i} new target with negative RTT")

dump_json(all_res, ANALYZABLE_FILE)

0 new target with negative RTT
